In [1]:
## parameters:
## run describe on the input table
describe = False
## save the result of the preprocessing pipeline as a table.
save_as_table = True

## for testing - pipeline will include categorical variables 0:n_sparse_features in the features table to estimate the model.
n_sparse_features = 0  
## for testing - pipeline will include numeric variables 0:n_num_features in the features table to estimate the model.
n_num_features = 5
classifier_lightgbm_iterations = 3  
n_folds = 4                         
num_leaves_grid = [2, 4, 8]

## load data loader

In [3]:
## from reco_utils.dataset.criteo_dac import load_spark_df

In [4]:
## read in the data - this takes some time...8-10 minutes
## df = load_spark_df(spark=spark, dbutils=dbutils)
df = sqlContext.read.parquet("/FileStore/dac_train.parquet")
# Could ADLS be causing issues?
# df = sqlContext.read.parquet("/mnt/adlsgen2/dac_train.parquet")

In [5]:
if describe:
  ## This can take quite a bit of time...
  cur_descr = df.describe()
  display(cur_descr)

In [6]:
## boundary check n_sparse_features
if n_sparse_features < 0 or n_sparse_features > 26:
  raise ValueError('n_sparse_features must be between 0 and 26...')
else:
  print('Running with {} sparse (i.e. categorical) features.'.format(n_sparse_features))
  
if n_num_features < 0 or n_num_features > 13:
  raise ValueError('n_sparse_features must be between 0 and 26...')
else:
  print('Running with {} numeric features.'.format(n_num_features))
  
if n_num_features+n_sparse_features < 1:
  raise ValueError('total number of features is less than 1.')

Running with 0 sparse (i.e. categorical) features.
Running with 5 numeric features.

## Imports

In [8]:
## for feature engineering:
from pyspark.ml.feature import (Imputer,StringIndexer,VectorAssembler)
from pyspark.ml.pipeline import Pipeline

## for modeling:
from mmlspark import LightGBMClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator


## Define what features to process

- `features` maps to numeric features that need to have missing values replaced
- `sparse_features` maps to the first `n_sparse_features` categorical / string variables

In [10]:

## features are int features (does median imputation)
features = [x for x in df.columns if x[0:3] == 'int'][0:n_num_features]
## sparse_features are str features 
sparse_features = [x for x in df.columns if x[0:3] == 'cat'][0:n_sparse_features]


## Recast `int` variables to `float`

`Imputer()` only works with `float` or `double` type. We could import the data as floats, or run directly on ints using the `df.na.fill()` method.

Currently using this approach to keep the work in the pipeline.

In [12]:
## cast ints to floats, because Imputer only works with floats
sql_lst = ['cast({} as float) {}'.format(x, x) for x in features] + sparse_features + ['label']
recast_df = df.selectExpr(*[sql_lst])

In [13]:
pipeline = Pipeline(stages=[
  Imputer(strategy='median',
          inputCols=features,
          outputCols=[f + '_imp' for f in features]),
  # LightGBM can handle categoricals directly if StringIndexer is used through meta-data
  *[StringIndexer(inputCol=f , outputCol=f+'_vec') for f in sparse_features],
  VectorAssembler(inputCols= [f + '_imp' for f in features] +
                  [f + '_vec' for f in sparse_features],
                  outputCol='features')
])

In [14]:
# fit is needed if you use imputer..
train_proc_df = pipeline.fit(recast_df).transform(recast_df)

In [15]:
table_to_save = 'criteo_dac_proc_{}sparse_{}num'.format(n_sparse_features,n_num_features)

try:
  train_proc_df.write.saveAsTable(table_to_save)
except:
  pass

## Set up the Classifier:

In [17]:
model = LightGBMClassifier(featuresCol='features',
                           labelCol='label',
                           numIterations=classifier_lightgbm_iterations,
                           numLeaves=8,
                           isUnbalance=True)

grid = (ParamGridBuilder()
        .addGrid(model.numLeaves, num_leaves_grid) 
        .build())

evaluator = BinaryClassificationEvaluator(labelCol='label')

cv = CrossValidator(estimator=model, estimatorParamMaps=grid, evaluator=evaluator, numFolds=n_folds)

## Fit the model.

In [19]:
## try just fitting the model, not with CV
## model fit works, sometimes.
model_fit = model.fit(train_proc_df)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1982148294690641> in <module> () 
 1 ## try just fitting the model, not with CV 
 2 ## model fit works, sometimes. 
 ----> 3 model_fit = model . fit ( train_proc_df ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2148.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 9.0 failed 4 times, most recent failure: Lost task 6.3 in stage 9.0 (TID 98, 10.139.64.9, executor 3): java.net.ConnectException: Connection refused (Connection refused)
	at java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:350)
	at java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocketImpl.java:206)
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:188)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:589)
	at java.net.Socket.connect(Socket.java:538)
	at java.net.Socket.<init>(Socket.java:434)
	at java.net.Socket.<init>(Socket.java:211)
	at com.microsoft.ml.spark.TrainUtils$.getNodes(TrainUtils.scala:178)
	at com.microsoft.ml.spark.TrainUtils$$anonfun$5.apply(TrainUtils.scala:211)
	at com.microsoft.ml.spark.TrainUtils$$anonfun$5.apply(TrainUtils.scala:205)
	at com.microsoft.ml.spark.StreamUtilities$.using(StreamUtilities.scala:29)
	at com.microsoft.ml.spark.TrainUtils$.trainLightGBM(TrainUtils.scala:204)
	at com.microsoft.ml.spark.LightGBMClassifier$$anonfun$3.apply(LightGBMClassifier.scala:83)
	at com.microsoft.ml.spark.LightGBMClassifier$$anonfun$3.apply(LightGBMClassifier.scala:83)
	at org.apache.spark.sql.execution.MapPartitionsExec$$anonfun$6.apply(objects.scala:200)
	at org.apache.spark.sql.execution.MapPartitionsExec$$anonfun$6.apply(objects.scala:197)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:852)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:852)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:60)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:340)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:304)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala

In [20]:
## estimate the model:
## throws an error:
cv_fit = cv.fit(train_proc_df)


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3150017522451508> in <module> () 
 1 ## estimate the model: 
 2 ## throws an error: 
 ----> 3 cv_fit = cv . fit ( train_proc_df ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 305 
 306 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 307 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 308 metrics [ j ] += ( metric / nFolds ) 
 309 metrics_all [ i ] [ j ] = metric 

 /usr/lib/python3.5/multiprocessing/pool.py in next (self, timeout) 
 693 if success : 
 694 return value 
 --> 695 raise value 
 696 
 697 __next__ = next # XXX 

 /usr/lib/python3.5/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 117 job , i , func , args , kwds = task 
 118 try : 
 --> 119 result = ( True , func ( * args , ** kwds ) ) 
 120 except Exception as e : 
 121 if wrap_exception : 

 /databricks/spark/python/pyspark/ml/tuning.py in <lambda> (f) 
 305 
 306 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 307 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 308 metrics [ j ] += ( metric / nFolds ) 
 309 metrics_all [ i ] [ j ] = metric 

 /databricks/spark/python/pyspark/ml/tuning.py in singleTask () 
 52 
 53 def singleTask ( ) : 
 ---> 54 index , model = next ( modelIter ) 
 55 metric = eva . evaluate ( model . transform ( validation , epm [ index ] ) ) 
 56 return index , metric , model if collectSubModel else None 

 /databricks/spark/python/pyspark/ml/base.py in __next__ (self) 
 60 raise StopIteration ( "No models remaining." ) 
 61 self . counter += 1 
 ---> 62 return index , self . fitSingleModel ( index ) 
 63 
 64 def next ( self ) : 

 /databricks/spark/python/pyspark/ml/base.py in fitSingleModel (index) 
 104 
 105 def fitSingleModel ( index ) : 
 --> 106 return estimator . fit ( dataset , paramMaps [ index ] ) 
 107 
 108 return _FitMultipleIterator ( fitSingleModel , len ( paramMaps ) ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 128 elif isinstance ( params , dict ) : 
 129 if params : 
 --> 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 132 return self . _fit ( dataset ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(